In [3]:
!uv pip install --system colpali_engine>=0.3.1 datasets huggingface_hub[hf_transfer] qdrant-client transformers>=4.45.0 stamina rich

Using Python 3.11.10 environment at /home/jinaai/miniconda3/envs/rerankvlm_datagen_cpi_1_2
Resolved 84 packages in 55.66s                                       
⠙ Preparing packages... (0/23)                                                  
⠙ Preparing packages... (0/23)-------     0 B/85.48 KiB                 
tenacity   ------------------------------     0 B/27.51 KiB
⠙ Preparing packages... (0/23)-------     0 B/85.48 KiB                 
annotated-types ------------------------------     0 B/13.32 KiB
tenacity   ------------------------------     0 B/27.51 KiB
⠙ Preparing packages... (0/23)-------     0 B/85.48 KiB                 
sniffio    ------------------------------     0 B/10.00 KiB
annotated-types ------------------------------     0 B/13.32 KiB
tenacity   ------------------------------     0 B/27.51 KiB
⠙ Preparing packages... (0/23)-------     0 B/85.48 KiB                 
mdurl      ------------------------------     0 B/9.75 KiB
sniffio    --------------------------

In [1]:
import os
import torch
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.http import models
from tqdm import tqdm
from datasets import Dataset, load_dataset, Features, Value, Sequence, Image, load_from_disk
from typing import List, cast

/home/jinaai/miniconda3/envs/rerankvlm_datagen_cpi_1_2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = cast(Dataset, load_dataset('/data/hf_datasets/colpali_train_set/', split="train"))

In [3]:
dataset_test = cast(Dataset, load_dataset('/data/hf_datasets/colpali_train_set/', split="test"))
dataset_test

Dataset({
    features: ['image', 'image_filename', 'query', 'answer', 'source', 'options', 'page', 'model', 'prompt', 'answer_type'],
    num_rows: 500
})

In [7]:
from colpali_engine.models import ColPali, ColPaliProcessor
from colpali_engine.utils.processing_utils import BaseVisualRetrieverProcessor

In [3]:
!export http_proxy=http://127.0.0.1:7890
!curl google.com

^C


In [12]:
!unset http_proxy
!curl baidu.com

<html>
<meta http-equiv="refresh" content="0;url=http://www.baidu.com/">
</html>


In [14]:
model_name = "vidore/colpali-v1.2"
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
# Load model
model = ColPali.from_pretrained(
    model_name, 
    cache_dir="/data/hf_models/",
    torch_dtype=torch.bfloat16, 
    device_map="cuda:0"
)
processor = ColPaliProcessor.from_pretrained(model_name)
model = model.eval()

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.40it/s]


ConnectionError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /vidore/colpali-v1.2/resolve/main/adapter_model.safetensors (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f67406f9d10>: Failed to establish a new connection: [Errno 101] Network is unreachable'))"), '(Request ID: a3a7568e-11a4-48fe-a13d-958fa348f4e2)')

In [ ]:
sample = dataset[:4]

from torch.utils.data import DataLoader
image_dataloader = DataLoader(
    dataset=sample["image"],
    batch_size=4,#config['batch_size']['image'],
    shuffle=False,
    collate_fn=lambda x: processor.process_images(x),
)

image_embeddings: List[torch.Tensor] = []
for batch_doc in tqdm(image_dataloader):
    with torch.no_grad():
        batch_doc = {k: v.to(model.device) for k, v in batch_doc.items()}
        embeddings_doc = model(**batch_doc)
    image_embeddings.extend(list(torch.unbind(embeddings_doc.to("cpu"))))

In [ ]:
print(embeddings_doc.shape)
vector_size = embeddings_doc.shape[2]
vector_size

In [ ]:
qdrant_client = QdrantClient(
    "/data/hf_datasets/qdrantdb"
)  # Use ":memory:" for in-memory database or "path/to/db" for persistent storage

In [ ]:
collection_name = "colpali_test"
vector_params = models.VectorParams(
    size=vector_size,
    distance=models.Distance.COSINE,
    multivector_config=models.MultiVectorConfig(
        comparator=models.MultiVectorComparator.MAX_SIM
    ),
)

In [ ]:
scalar_quant = models.ScalarQuantizationConfig(
    type=models.ScalarType.INT8,
    quantile=0.99,
    always_ram=False,
)

In [ ]:
qdrant_client.recreate_collection(
    collection_name=collection_name,  # the name of the collection
    on_disk_payload=True,  # store the payload on disk
    optimizers_config=models.OptimizersConfigDiff(
        indexing_threshold=100
    ),  # it can be useful to swith this off when doing a bulk upload and then manually trigger the indexing once the upload is done
    vectors_config=models.VectorParams(
        size=vector_size,
        distance=models.Distance.COSINE,
        multivector_config=models.MultiVectorConfig(
            comparator=models.MultiVectorComparator.MAX_SIM
        ),
        quantization_config=models.ScalarQuantization(
            scalar=models.ScalarQuantizationConfig(
                type=models.ScalarType.INT8,
                quantile=0.99,
                always_ram=True,
            ),
        ),
    ),
)